## BEFORE YOU BEGIN

1. create a directory called 'raw_data' in the directory containing this notebook
2. add the data files to in `./raw_data` with the name "RAW_<type_of_data>.csv"
  - recipes: `RAW_recipes.csv`
    - https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions/data?select=RAW_recipes.csv
  - interactions: `RAW_interactions.csv`
    - https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions/data?select=RAW_interactions.csv
  - ingredients: `RAW_ingredients.csv`
    - https://www.kaggle.com/datasets/thedevastator/the-nutritional-content-of-food-a-comprehensive
  - allergens: `RAW_allergens.csv`
    - https://www.kaggle.com/datasets/boltcutters/food-allergens-and-allergies 
3. create a directory called 'cleaned_data' in the directory containing this notebook, this is where the output csvs will end up

In [246]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
import ast
import re

recipes = pd.read_csv('./raw_data/RAW_recipes.csv')
recipes['ingredients'] = recipes['ingredients'].apply(ast.literal_eval)

## Cleaning Ingredients

In [247]:
ingredients = pd.read_csv('./raw_data/RAW_ingredients.csv')
ingredients['Shrt_Desc'] = ingredients['Shrt_Desc'].str.lower().apply(lambda x: re.sub(r'\s*,\s*', ' ', x))
basics = {"onion": "onions", "apples": "gala apples", "cream": "whipping cream", "apple": "gala apple", "nuts": "mixed nuts", "whipping cream": "heavy whipping cream", "strawberry": "strawberries", "oil": "canola oil", "milk": "whole milk", "yogurt": "yogurt plain", "flour": "all-purpose flour", "sugar": "granulated sugar", "pepper": "black pepper", "eggs": "whole egg", "egg": "whole egg", "butter": "unsalted butter", "tomato": "red tomato", "tomatoes": "red tomatoes"}

In [248]:
ingredients

,index,NDB_No,Shrt_Desc,Water_(g),Energ_Kcal,Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),...,Vit_K_(µg),FA_Sat_(g),FA_Mono_(g),FA_Poly_(g),Cholestrl_(mg),GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Refuse_Pct
0,0,1001,butter with salt,15.87,717,0.85,81.11,2.11,0.06,0.0,...,7.0,51.368,21.021,3.043,215.0,5.00,"1 pat, (1"" sq, 1/3"" high)",14.2,1 tbsp,0.0
1,1,1002,butter whipped w/ salt,16.72,718,0.49,78.30,1.62,2.87,0.0,...,4.6,45.390,19.874,3.331,225.0,3.80,"1 pat, (1"" sq, 1/3"" high)",9.4,1 tbsp,0.0
2,2,1003,butter oil anhydrous,0.24,876,0.28,99.48,0.00,0.00,0.0,...,8.6,61.924,28.732,3.694,256.0,12.80,1 tbsp,205.0,1 cup,0.0
3,3,1004,cheese blue,42.41,353,21.40,28.74,5.11,2.34,0.0,...,2.4,18.669,7.778,0.800,75.0,28.35,1 oz,17.0,1 cubic inch,0.0
4,4,1005,cheese brick,41.11,371,23.24,29.68,3.18,2.79,0.0,...,2.5,18.764,8.598,0.784,94.0,132.00,"1 cup, diced",113.0,"1 cup, shredded",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8785,8785,83110,mackerel salted,43.00,305,18.50,25.10,13.40,0.00,0.0,...,7.8,7.148,8.320,6.210,95.0,80.00,"1 piece, (5-1/2"" x 1-1/2"" x 1/2"")",17.0,"1 cubic inch, boneless",0.0
8786,8786,90240,scallop (bay&sea) ckd stmd,70.25,111,20.54,0.84,2.97,5.41,0.0,...,0.0,0.218,0.082,0.222,41.0,85.00,3 oz,NaN,NaN,0.0
8787,8787,90480,syrup cane,26.00,269,0.00,0.00,0.86,73.14,0.0,...,0.0,0.000,0.000,0.000,0.0,21.00,1 serving,NaN,NaN,0.0
8788,8788,90560,snail raw,79.20,90,16.10,1.40,1.30,2.00,0.0,...,0.1,0.361,0.259,0.252,50.0,85.00,3 oz,NaN,NaN,0.0


In [249]:
remove_keywords = ["sugar sub", "entree", "ckd", "infant formula", "inf formula", "child formula", "meat only", "skin only", "applebee's", "t.g.i friday's", "restaurant", "denny's", "ckd", "babyfood", "fast foods", "burger king", "mcdonald's", "wendy's", "pizza hut", "kfc", "taco bell", "subway", "domino's", "papa john's", "little caesars", "chick-fil-a", "popeyes", "olive garden", "on the border", "carraba's", "cracker barrel"]
replace_raw = {'pork cured bacon': "bacon", 'commly prep': "", 'no chol': "", 'unenr': "", 'raw': "", 'frsh': "fresh", 'frz': "frozen", 'crm': "cream", 'pnut': "peanut", 'pln': "plain", 'whl': "whole", 'dk': "dark", 'broilers or fryers': "", 'drsng': "dressing", 'spread': "spread", 'flr': "flour", 'bf': "beef" ,'w/': "with", 'wo/': "without", 'hvy': "heavy", 'lt': "light", 'juc': "juice", 'wo/salt': "unsalted", 'wo/ salt': "unsalted", 'without salt': "unsalted", "salad or cooking": "", 'yel': "yellow", 'sau': "sauce", 'veg': "vegetable"}

mask = ingredients['Shrt_Desc'].str.contains('|'.join(remove_keywords))
ingredients = ingredients[~mask]

replace = {f'^{key}\\b': value + " " for key, value in replace_raw.items()}
replace.update({f'\\b{key}$': " " + value for key, value in replace_raw.items()})
replace.update({f'\\b{key}\\b': " " + value + " " for key, value in replace_raw.items()})
replace.update({'\\bmilkfat\\b.*$': ""})
replace.update({'\\((.*?)\\)': ""})

ingredients.reset_index(drop=True, inplace=True)
ingredients["Cleaned_Desc"] = ingredients["Shrt_Desc"].replace(replace, regex=True).replace(r'\s+', ' ', regex=True).str.strip()
ingredient_names = ingredients['Cleaned_Desc'].unique()

ingredient_names

C:\Users\aprin\AppData\Local\Temp\ipykernel_17172\1717127808.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingredients["Cleaned_Desc"] = ingredients["Shrt_Desc"].replace(replace, regex=True).replace(r'\s+', ' ', regex=True).str.strip()


array(['butter with salt', 'butter whipped w/ salt',
       'butter oil anhydrous', ..., 'syrup cane', 'snail', 'turtle green'],
      dtype=object)

In [250]:
def matchWord(target, tomatch):
  l = list(map(lambda x : x[0], process.extractBests(target, tomatch, scorer=fuzz.token_sort_ratio, score_cutoff=50, limit=20)))
  return process.extractOne(target, l, scorer=fuzz.token_set_ratio)

### Scraper to get ingredient quantities from Food.com

In [251]:
from bs4 import BeautifulSoup
from fractions import Fraction
import requests

ingredient_quantity_map = {
    'cups': ['cup', 'cups'],
    'teaspoons': ['teaspoon', 'teaspoons', 'tsp', 'tsps'],
    'tablespoons': ['tablespoon', 'tablespoons', 'tbsp', 'tbsps'],
    'pints': ['pint', 'pints', 'pt', 'pts'],
    'quarts': ['quart', 'quarts', 'qt', 'qts'],
    'gallons': ['gallon', 'gallons', 'gal', 'gals'],
    'milliliters': ['milliliter', 'milliliters', 'ml', 'mls'],
    'liters': ['liter', 'liters'],
    'grams': ['gram', 'grams', 'g'],
    'kilograms': ['kilogram', 'kilograms', 'kg', 'kgs'],
    'ounces': ['ounce', 'ounces', 'oz'],
    'pounds': ['pound', 'pounds', 'lb', 'lbs'],
}

quantity_map = {value: key for key, values in ingredient_quantity_map.items() for value in values}

url_base = "https://www.food.com/recipe/"

def delete_and_everything_after(string, substring):
  return string.partition(substring)[0]

def get_after(string, substring):
  index = string.find(substring)
  return string[index + len(substring):] if index != -1 else string

def match_quantity_type(input_string):
  if input_string == "":
    return None

  choices = quantity_map.keys()

  best_match, similarity = process.extractOne(input_string, choices)

  return quantity_map[best_match] if similarity >= 90 else None
  
def parse_mixed_fraction(mixed_fraction):
  parts = mixed_fraction.split()

  if len(parts) == 1:
    # If there is no space, it's just a whole number or a fraction
    return float(Fraction(parts[0]))
  elif len(parts) == 2:
    # If there is a space, it's a mixed fraction
    whole_number = float(parts[0])
    fraction_parts = parts[1].split('/')
    numerator = float(fraction_parts[0])
    denominator = float(fraction_parts[1]) if len(fraction_parts) > 1 else 1

    return whole_number + (numerator / denominator)
  else:
    raise ValueError("Invalid mixed fraction format")

def get_ingredient_quantities(recipe):
  url = url_base + recipe['name'].replace(" ", "-") + "-" + str(recipe['id'])
  page = requests.get(url)
  
  if page.status_code != 200:
    return None

  soup = BeautifulSoup(page.content, 'html.parser')
  
  if soup.find(class_='ingredient-list') is None:
    return None
  
  ingredient_list = soup.select('ul.ingredient-list li')
  
  if len(ingredient_list) != len(recipe['ingredients']):
    return None
  
  quantities = dict()
  for element, ingredient in list(zip(ingredient_list, recipe['ingredients'])):
    quantity_span = element.find('span', class_='ingredient-quantity')
    ingredient_quantity_match = re.compile(r'\d+\s+\d+\/\d+|\d+\/\d+|\d+').search(quantity_span.text) if quantity_span else None
    ingredient_quantity = ingredient_quantity_match.group() if ingredient_quantity_match else None


    text_span = element.find('span', class_='ingredient-text')
    ingredient_quantity_type = text_span.get_text(strip=True) if text_span else None
    
    if ingredient_quantity_type and re.compile(r'\d+\s+\d+\/\d+|\d+\/\d+|\d+').search(ingredient_quantity_type):
      ingredient_quantity = re.compile(r'\d+\s+\d+\/\d+|\d+\/\d+|\d+').search(ingredient_quantity_type).group()
      ingredient_quantity_type = get_after(ingredient_quantity_type, ingredient_quantity)

    quantity = parse_mixed_fraction(ingredient_quantity) if ingredient_quantity else None
    quantity_type = match_quantity_type(delete_and_everything_after(ingredient_quantity_type, ingredient)) if ingredient_quantity_type else None
    
    quantities[ingredient] = (quantity, quantity_type)
  return quantities

#### Get recipes that match cleaned ingredients

In [253]:
valid = dict()
valid_with_ingred = dict()
bad_match = dict()
no_match = []
for _, recipe in recipes.loc[:20000].iterrows():
  skip = False
  ingreds_in_recipe = dict()
  for ingred in recipe["ingredients"]:
    to_match = ingred
    if ingred in basics.keys():
      to_match = basics[ingred]

    res = matchWord(to_match, ingredient_names)
    if res is None:
      no_match.append(to_match)
      skip = True
      break
    match, percent = res
    ingreds_in_recipe[to_match] = match
    if percent < 90:
      bad_match[to_match] = (match, percent)
      skip = True
      break
  if not skip:
    quantity_info = get_ingredient_quantities(recipe)
    if quantity_info:
      result_dict = {key: (ingreds_in_recipe[key], quantity_info[key]) for key in set(ingreds_in_recipe) & set(quantity_info)}
      valid_with_ingred[recipe['name']] = (quantity_info)
      valid[recipe['name']] = result_dict
    skip = False
    

## Get needed ingredients

In [254]:
print("Valid recipes: ", len(valid))
unique_ingreds = list(set(ingred for recipe in valid.values() for ingred, _ in recipe.values()))
print("Unique ingredients in recipes: ", len(unique_ingreds))

mask = ingredients['Cleaned_Desc'].isin(unique_ingreds)
unneeded_cols = ["Shrt_Desc", "Refuse_Pct", "index"]
ingredients_working = ingredients[mask].drop(columns=unneeded_cols).drop_duplicates(subset=['Cleaned_Desc'], keep='first').reset_index(drop=True)
ingredients_working


Valid recipes:  1279
Unique ingredients in recipes:  527


,NDB_No,Water_(g),Energ_Kcal,Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),Sugar_Tot_(g),Calcium_(mg),...,Vit_K_(µg),FA_Sat_(g),FA_Mono_(g),FA_Poly_(g),Cholestrl_(mg),GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Cleaned_Desc
0,1004,42.41,353,21.40,28.74,5.11,2.34,0.0,0.50,528.0,...,2.4,18.669,7.778,0.800,75.0,28.35,1 oz,17.0,1 cubic inch,cheese blue
1,1006,48.42,334,20.75,27.68,2.70,0.45,0.0,0.45,184.0,...,2.3,17.410,8.013,0.826,100.0,28.35,1 oz,144.0,"1 cup, sliced",cheese brie
2,1007,51.80,300,19.80,24.26,3.68,0.46,0.0,0.46,388.0,...,2.0,15.259,7.023,0.724,72.0,28.35,1 oz,246.0,1 cup,cheese camembert
3,1009,37.02,404,22.87,33.31,3.71,3.09,0.0,0.48,710.0,...,2.4,18.867,9.246,1.421,99.0,132.00,"1 cup, diced",244.0,"1 cup, melted",cheese cheddar
4,1015,81.24,81,10.45,2.27,1.27,4.76,0.0,4.00,111.0,...,0.0,1.235,0.516,0.083,12.0,113.00,4 oz,226.0,"1 cup, (not packed)",cheese cottage lowfat 2%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,43392,69.50,115,2.70,0.20,1.99,25.61,1.5,17.16,18.0,...,0.0,0.046,0.005,0.089,0.0,NaN,NaN,NaN,NaN,hearts of palm
523,43572,2.80,429,12.60,9.50,1.71,73.39,14.2,0.54,11.0,...,15.7,1.415,4.085,3.572,0.0,28.35,1 oz,NaN,NaN,popcorn microwave lofat&na
524,43598,21.70,688,0.00,77.80,0.40,0.30,0.0,0.30,7.0,...,24.7,10.784,18.026,45.539,0.0,15.00,1 tbsp,239.0,1 cup,mayonnaise dressing
525,48052,8.20,370,75.16,1.85,1.00,13.79,0.6,0.00,142.0,...,0.0,0.272,0.156,0.810,0.0,NaN,NaN,NaN,NaN,vital wheat gluten


In [255]:
volume_units = ['cups', 'teaspoons', 'tablespoons', 'fluid ounces', 'pints', 'quarts', 'gallons', 'milliliters', 'liters']
weight_units = ['grams', 'kilograms', 'ounces', 'pounds']

def get_number (string):
  return re.compile(r'\d?\.\d+|\d+').search(string).group()
def something(ingredient):
  quantities = {'volume': {'quantity': None, 'desc': None}, 'weight': {'quantity': None, 'desc': None}, 'unit': None}
  gm1 = ingredient['GmWt_Desc1']
  gm2 = ingredient['GmWt_Desc2']
  
  if isinstance(gm1, str):
    gm1_match = match_quantity_type(gm1)
    gm1_num = get_number(gm1)
    if gm1_match in volume_units:
      quantities['volume'] = {'quantity': gm1_num, 'desc': gm1_match}
    elif gm1_match in weight_units:
      quantities['weight'] = {'quantity': gm1_num, 'desc': gm1_match}
    else:
      quantities['unit'] = gm1_num
    
  if isinstance(gm2, str):
    gm2_match = match_quantity_type(gm2)
    gm2_num = get_number(gm2)
    if gm2_match in volume_units:
      quantities['volume'] = {'quantity': gm2_num, 'desc': gm2_match}
    elif gm2_match in weight_units:
      quantities['weight'] = {'quantity': gm2_num, 'desc': gm2_match}
    else:
      quantities['unit'] = gm2_num
  return quantities
  
ingredients_working['quantities'] = ingredients_working.apply(something, axis=1)
ingredients_csv = ingredients_working.join(ingredients_working['quantities'].apply(pd.Series))
ingredients_csv = ingredients_csv.join(ingredients_csv['volume'].apply(pd.Series).add_prefix('volume_'))
ingredients_csv = ingredients_csv.join(ingredients_csv['weight'].apply(pd.Series).add_prefix('weight_'))

to_keep = ["NDB_No", "Energ_Kcal", "Protein_(g)", "Carbohydrt_(g)", "Fiber_TD_(g)", "Sugar_Tot_(g)", "Calcium_(mg)", "Iron_(mg)", "Magnesium_(mg)", "Sodium_(mg)", "Cholestrl_(mg)", "Cleaned_Desc", "unit", "volume_quantity", "volume_desc", "weight_quantity", "weight_desc"]
to_drop = set(ingredients_csv.columns) - set(to_keep)

ingredients_csv = ingredients_csv.drop(columns=to_drop)

ingredients_csv

,NDB_No,Energ_Kcal,Protein_(g),Carbohydrt_(g),Fiber_TD_(g),Sugar_Tot_(g),Calcium_(mg),Iron_(mg),Magnesium_(mg),Sodium_(mg),Cholestrl_(mg),Cleaned_Desc,unit,volume_quantity,volume_desc,weight_quantity,weight_desc
0,1004,353,21.40,2.34,0.0,0.50,528.0,0.31,23.0,1146.0,75.0,cheese blue,1,None,None,1,ounces
1,1006,334,20.75,0.45,0.0,0.45,184.0,0.50,20.0,629.0,100.0,cheese brie,None,1,cups,1,ounces
2,1007,300,19.80,0.46,0.0,0.46,388.0,0.33,20.0,842.0,72.0,cheese camembert,None,1,cups,1,ounces
3,1009,404,22.87,3.09,0.0,0.48,710.0,0.14,27.0,653.0,99.0,cheese cheddar,None,1,cups,None,None
4,1015,81,10.45,4.76,0.0,4.00,111.0,0.13,9.0,308.0,12.0,cheese cottage lowfat 2%,None,1,cups,4,ounces
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,43392,115,2.70,25.61,1.5,17.16,18.0,1.69,10.0,14.0,0.0,hearts of palm,None,None,None,None,None
523,43572,429,12.60,73.39,14.2,0.54,11.0,2.28,151.0,490.0,0.0,popcorn microwave lofat&na,None,None,None,1,ounces
524,43598,688,0.00,0.30,0.0,0.30,7.0,0.23,1.0,486.0,0.0,mayonnaise dressing,None,1,cups,None,None
525,48052,370,75.16,13.79,0.6,0.00,142.0,5.20,25.0,29.0,0.0,vital wheat gluten,None,None,None,None,None


##### Get conversion factors

In [256]:
conversion_factors = {
    'cups': {
        'cups': 1,
        'teaspoons': 48,
        'tablespoons': 16,
        'pints': 0.5,
        'quarts': 0.25,
        'gallons': 0.0625,
        'milliliters': 236.588,
        'liters': 0.236588,
        'grams': None,
        'kilograms': None,
        'ounces': 8,
        'pounds': 0.0625,
    },
    'teaspoons': {
        'cups': 1./48,
        'teaspoons': 1,
        'tablespoons': 1./3,
        'pints': 1./96,
        'quarts': 1./192,
        'gallons': 1./768,
        'milliliters': 5.91939,
        'liters': 0.00591939,
        'grams': None,
        'kilograms': None,
        'ounces': 1./6,
        'pounds': 1./48,
    },
    'tablespoons': {
        'cups': 1./16,
        'teaspoons': 3,
        'tablespoons': 1,
        'pints': 1./32,
        'quarts': 1./64,
        'gallons': 1./256,
        'milliliters': 14.79,
        'liters': 0.01479,
        'grams': None,
        'kilograms': None,
        'ounces': 0.5,
        'pounds': 1./32,
    },
    'pints': {
        'cups': 2,
        'teaspoons': 96,
        'tablespoons': 32,
        'pints': 1,
        'quarts': 0.5,
        'gallons': 0.125,
        'milliliters': 473.176,
        'liters': 0.473176,
        'grams': None,
        'kilograms': None,
        'ounces': 16,
        'pounds': 0.125,
    },
    'quarts': {
        'cups': 4,
        'teaspoons': 192,
        'tablespoons': 64,
        'pints': 2,
        'quarts': 1,
        'gallons': 0.25,
        'milliliters': 946.353,
        'liters': 0.946353,
        'grams': None,
        'kilograms': None,
        'ounces': 32,
        'pounds': 0.25,
    },
    'gallons': {
        'cups': 16,
        'teaspoons': 768,
        'tablespoons': 256,
        'pints': 8,
        'quarts': 4,
        'gallons': 1,
        'milliliters': 3785.41,
        'liters': 3.78541,
        'grams': None,
        'kilograms': None,
        'ounces': 128,
        'pounds': 1,
    },
    'milliliters': {
        'cups': 0.00422675,
        'teaspoons': 0.16907,
        'tablespoons': 0.0563121,
        'pints': 0.00211338,
        'quarts': 0.00105669,
        'gallons': 0.000264172,
        'milliliters': 1,
        'liters': 0.001,
        'grams': None,
        'kilograms': None,
        'ounces': 0.033814,
        'pounds': 0.00220462,
    },
    'liters': {
        'cups': 4.22675,
        'teaspoons': 202.884,
        'tablespoons': 67.628,
        'pints': 2.11338,
        'quarts': 1.05669,
        'gallons': 0.264172,
        'milliliters': 1000,
        'liters': 1,
        'grams': None,
        'kilograms': None,
        'ounces': 33.814,
        'pounds': 2.20462,
    },
    'grams': {
        'cups': None,
        'teaspoons': None,
        'tablespoons': None,
        'pints': None,
        'quarts': None,
        'gallons': None,
        'milliliters': None,
        'liters': None,
        'grams': 1,
        'kilograms': 0.001,
        'ounces': 0.03527396,
        'pounds': 0.00220462,
    },
    'kilograms': {
        'cups': None,
        'teaspoons': None,
        'tablespoons': None,
        'pints': None,
        'quarts': None,
        'gallons': None,
        'milliliters': None,
        'liters': None,
        'grams': 1000,
        'kilograms': 1,
        'ounces': 35.27396,
        'pounds': 2.20462,
    },
    'ounces': {
        'cups': 0.125,
        'teaspoons': 6,
        'tablespoons': 2,
        'pints': 0.0625,
        'quarts': 0.03125,
        'gallons': 0.0078125,
        'milliliters': 29.5735,
        'liters': 0.0295735,
        'grams': 28.3495,
        'kilograms': 0.0283495,
        'ounces': 1,
        'pounds': 0.0625,
    },
    'pounds': {
        'cups': 16,
        'teaspoons': 768,
        'tablespoons': 256,
        'pints': 8,
        'quarts': 4,
        'gallons': 1,
        'milliliters': 453592,
        'liters': 453.592,
        'grams': 453.592,
        'kilograms': 0.453592,
        'ounces': 16,
        'pounds': 1,
    }
}

conversions = [{"from": key, "to": subkey, "factor": value} for key, sub_dict in conversion_factors.items() for subkey, value in sub_dict.items()]

conversions_csv = pd.DataFrame(conversions).dropna()

conversions_csv

,from,to,factor
0,cups,cups,1.000000
1,cups,teaspoons,48.000000
2,cups,tablespoons,16.000000
3,cups,pints,0.500000
4,cups,quarts,0.250000
...,...,...,...
139,pounds,liters,453.592000
140,pounds,grams,453.592000
141,pounds,kilograms,0.453592
142,pounds,ounces,16.000000


## Get valid recipes

In [257]:
valid_recipe_names = list(valid.keys())

recipes_working = recipes[recipes['name'].isin(valid_recipe_names)].drop_duplicates(subset=['name'], keep='first').reset_index(drop=True)

recipes_working['steps'] = recipes_working['steps'].apply(ast.literal_eval)

recipes_working

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,better than sex strawberries,42198,1460,41531,2002-10-03,"['weeknight', 'time-to-make', 'course', 'main-...","[734.1, 66.0, 199.0, 10.0, 10.0, 117.0, 28.0]",8,[crush vanilla wafers into fine crumbs and lin...,simple but sexy. this was in my local newspape...,"[vanilla wafers, butter, powdered sugar, eggs,...",7
1,chinese chop suey,8559,70,4481,2001-01-27,"['weeknight', 'time-to-make', 'course', 'main-...","[395.4, 31.0, 20.0, 29.0, 51.0, 33.0, 8.0]",8,"[brown ground meat and onion in a large pot, a...",easy one-pot dinner.,"[celery, onion, ground pork, soy sauce, beef b...",7
2,fried potatoes,37073,40,1533,2002-08-13,"['60-minutes-or-less', 'time-to-make', 'course...","[132.6, 8.0, 4.0, 3.0, 4.0, 5.0, 6.0]",14,"[preheat oven to 400 degrees, cut the potatoes...","my husband made these up last week, very tasty...","[red potatoes, margarine, rosemary]",3
3,momma s special marinade,30131,10,41480,2002-06-03,"['15-minutes-or-less', 'time-to-make', 'course...","[199.2, 31.0, 6.0, 10.0, 1.0, 13.0, 1.0]",4,[depending on how much meat i have i usually h...,we usually use with chicken. my neices ask for...,"[lemon juice, oil, worcestershire sauce, basil...",7
4,munch without guilt tomatoes,30300,10,6164,2002-06-04,"['15-minutes-or-less', 'time-to-make', 'course...","[3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",3,[put a slice of tomato on each biscuit / crack...,anytime munchies...another sweat free preparat...,"[tomatoes, crackers, mayonnaise, black pepper]",4
...,...,...,...,...,...,...,...,...,...,...,...,...
1274,bed and breakfast cottage cheese pancakes,348917,18,37449,2009-01-11,"['30-minutes-or-less', 'time-to-make', 'course...","[362.6, 42.0, 6.0, 17.0, 27.0, 69.0, 4.0]",12,"[separate eggs, beat whites till you have stif...",adapted from the churchill house(bed and break...,"[cottage cheese, flour, wheat germ, butter, eg...",6
1275,bed and breakfast rolls,223742,30,129958,2007-04-20,"['30-minutes-or-less', 'time-to-make', 'course...","[105.4, 8.0, 22.0, 5.0, 3.0, 12.0, 3.0]",12,"[cream together cream cheese , mayonnaise and ...","this recipe comes from my ""no time to cook"" co...","[cream cheese, mayonnaise, milk, white bread, ...",7
1276,bedouin fresh date sweet rangina gluten free,405481,17,593927,2009-12-29,"['30-minutes-or-less', 'time-to-make', 'course...","[725.5, 80.0, 126.0, 19.0, 7.0, 164.0, 21.0]",8,[remove pits from the dates and arrange on sma...,a simple delicious gulf arabian sweet. perfect...,"[dates, butter, rice flour, cardamom]",4
1277,beef chimichurri sauce,307536,26,386585,2008-06-05,"['30-minutes-or-less', 'time-to-make', 'course...","[134.5, 20.0, 0.0, 12.0, 1.0, 9.0, 0.0]",13,"[in a food processor , combine parsley , shall...",summer is the perfect time to make this argent...,"[fresh parsley leaves, shallot, garlic cloves,...",8


##### Get recipe steps

In [258]:
steps_csv = recipes_working.explode('steps')

steps_csv['order'] = steps_csv.groupby('id').cumcount() + 1
steps_csv = steps_csv.reset_index(drop=True)

to_keep = ['id', 'steps', 'order']
to_drop = set(steps_csv) - set(to_keep)

steps_csv = steps_csv.drop(columns=to_drop)

steps_csv

,id,steps,order
0,42198,crush vanilla wafers into fine crumbs and line...,1
1,42198,mix butter or margarine and sugar,2
2,42198,add beaten eggs,3
3,42198,spread the mixture over the wafer crumbs,4
4,42198,"crush strawberries and spread over sugar , egg...",5
...,...,...,...
9376,182042,stir,6
9377,182042,spread over rice in dish,7
9378,182042,pour the chicken broth over the mixture,8
9379,182042,cover and bake at 350 for 1 hour,9


### Get final recipes data

In [259]:
recipes_working

to_keep = ["name", "id", "minutes", "contributor_id", "submitted", "tags", "description", "nutrition"]
to_drop = set(recipes_working.columns) - set(to_keep)

recipes_csv = recipes_working.drop(columns=to_drop)
recipes_csv['calories'] = recipes_working['nutrition'].apply(ast.literal_eval).apply(lambda arr: arr[0])
recipes_csv = recipes_csv.drop(columns=['nutrition'])

recipes_csv

,name,id,minutes,contributor_id,submitted,tags,description,calories
0,better than sex strawberries,42198,1460,41531,2002-10-03,"['weeknight', 'time-to-make', 'course', 'main-...",simple but sexy. this was in my local newspape...,734.1
1,chinese chop suey,8559,70,4481,2001-01-27,"['weeknight', 'time-to-make', 'course', 'main-...",easy one-pot dinner.,395.4
2,fried potatoes,37073,40,1533,2002-08-13,"['60-minutes-or-less', 'time-to-make', 'course...","my husband made these up last week, very tasty...",132.6
3,momma s special marinade,30131,10,41480,2002-06-03,"['15-minutes-or-less', 'time-to-make', 'course...",we usually use with chicken. my neices ask for...,199.2
4,munch without guilt tomatoes,30300,10,6164,2002-06-04,"['15-minutes-or-less', 'time-to-make', 'course...",anytime munchies...another sweat free preparat...,3.0
...,...,...,...,...,...,...,...,...
1274,bed and breakfast cottage cheese pancakes,348917,18,37449,2009-01-11,"['30-minutes-or-less', 'time-to-make', 'course...",adapted from the churchill house(bed and break...,362.6
1275,bed and breakfast rolls,223742,30,129958,2007-04-20,"['30-minutes-or-less', 'time-to-make', 'course...","this recipe comes from my ""no time to cook"" co...",105.4
1276,bedouin fresh date sweet rangina gluten free,405481,17,593927,2009-12-29,"['30-minutes-or-less', 'time-to-make', 'course...",a simple delicious gulf arabian sweet. perfect...,725.5
1277,beef chimichurri sauce,307536,26,386585,2008-06-05,"['30-minutes-or-less', 'time-to-make', 'course...",summer is the perfect time to make this argent...,134.5


## Make recipe-ingredient relations

In [260]:
def find_recipe_id(recipe_name):
  return recipes_csv[recipes_csv['name'] == recipe_name]['id'].item()

def find_ingredient_id(ingredient_name):
  return ingredients_csv[ingredients_csv['Cleaned_Desc'] == ingredient_name]["NDB_No"].item()

def create_relationship(recipe_name, ingredients):
  recipe_id = find_recipe_id(recipe_name)
  ingred_rows = [{"recipe_id": recipe_id, "ingredient_id": find_ingredient_id(name), "quantity": quantity, "quantity_desc": type, "variant": key} for key, (name, (quantity, type)) in ingredients.items()]
  return ingred_rows

relation_temp = [create_relationship(recipe_name, ingredients) for recipe_name, (ingredients) in valid.items()]
ingredient_in_csv = pd.DataFrame([row for rows in relation_temp for row in rows])

ingredient_in_csv

,recipe_id,ingredient_id,quantity,quantity_desc,variant
0,42198,19336,16.0,ounces,powdered sugar
1,42198,18609,12.0,ounces,vanilla wafers
2,42198,12155,NaN,None,walnuts
3,8559,28387,6.0,None,hamburger buns
4,8559,11143,1.0,None,celery
...,...,...,...,...,...
5786,182042,2047,2.0,teaspoons,salt
5787,182042,20036,1.0,None,long-grain rice
5788,182042,13047,1.0,pounds,ground beef
5789,182042,11109,1.0,None,head of cabbage


## Interactions

In [261]:
interactions = pd.read_csv('./raw_data/RAW_interactions.csv')
interactions_csv = interactions[interactions['recipe_id'].isin(recipes_csv['id'])].reset_index(drop=True)
interactions_csv

,user_id,recipe_id,date,rating,review
0,52476,181765,2006-08-15,5,We have been eating this bbq sauce for years! ...
1,37722,181765,2007-03-12,5,This is excellent and the same recipe I've alw...
2,135470,181765,2008-05-05,1,I did not care for this at all. All I could ta...
3,286566,352182,2011-02-16,4,We really liked the marinade but next time I w...
4,1802964648,424551,2014-07-25,4,When making this recipe be aware of the scovil...
...,...,...,...,...,...
8402,5060,39370,2002-11-24,5,"WOW!! Chris, made this for Dad and me for the ..."
8403,169430,190261,2007-01-14,5,"WOW, these are delicious! I used anise seed th..."
8404,58104,190261,2007-06-07,5,Tasty! A nice hint of anise and sweetness from...
8405,1197076,190261,2010-03-08,0,Delicious ! I tweeked the recipe a bit>>substi...


## Create User Data

In [262]:
user_ids = pd.concat([interactions_csv['user_id'], recipes_csv['contributor_id']], ignore_index=True).unique()
print("Unique users: ", len(user_ids))

Unique users:  5754


In [263]:
from faker import Faker

fake = Faker()

In [264]:
def generate_info():
  name = fake.first_name() + " " + fake.last_name()
  domain = fake.free_email_domain()
  split_name = name.lower().split(" ")
  rand = fake.random_int(min=0, max=100) % 7
  email = ""
  if rand == 1:
    email = split_name[0] + split_name[1] + "@" + domain
  elif rand == 2:
    email = split_name[0] + "." + split_name[1] + "@" + domain
  elif rand == 3:
    email = split_name[0][0] + split_name[1] + str(fake.random_int(min=1, max=100)) + "@" + domain
  elif rand == 4:
    email = split_name[0] + str(fake.random_int(min=1, max=100)) + "@" + domain
  elif rand == 5:
    email = split_name[1] + split_name[0] + "@" + domain
  elif rand == 6:
    email = split_name[1] + "." + split_name[0] + "@" + domain
  else:
    email = split_name[0] + "_" + split_name[1] + str(fake.random_int(min=1, max=100)) + "@" + domain
  return name, email
    

users_csv = pd.DataFrame([{"id": user_id, "name": name, "email": email} for user_id, (name, email) in zip(user_ids, [generate_info() for _ in user_ids])])

users_csv

,id,name,email
0,52476,Caroline Ramos,caroline_ramos20@hotmail.com
1,37722,Sarah Wolfe,sarah.wolfe@yahoo.com
2,135470,Blake Lee,blake_lee78@yahoo.com
3,286566,Mark Taylor,mark.taylor@yahoo.com
4,1802964648,Alicia Carlson,acarlson53@yahoo.com
...,...,...,...
5749,90633,Katherine Taylor,taylorkatherine@hotmail.com
5750,1433633,Chad Sandoval,chad33@gmail.com
5751,2602981,William Mathews,william.mathews@gmail.com
5752,121041,Jennifer Davis,jennifer.davis@hotmail.com


## Allergen Data

In [265]:
allergies_to_consider = ["nut", "lactose"]
allergens = pd.read_csv('./raw_data/RAW_allergens.csv')

allergens['Allergy'] = allergens['Allergy'].str.lower()
allergens['Food'] = allergens['Food'].str.lower()

allergens.dropna(subset=['Allergy'], inplace=True)

allergen_keywords = {allergen : allergens[allergens['Allergy'].str.contains(allergen)]["Food"].values for allergen in allergies_to_consider}

allergen_keywords

{'nut': array(['almond', 'chestnut', 'ginkgo nut', 'peanut', 'pecan', 'walnut'],
       dtype=object),
 'lactose': array(['butter', 'buttermilk', 'casein', 'cheese', 'cream', 'custard',
        'ice cream', 'lactose', 'milk', 'sour cream', 'whey', 'yogurt '],
       dtype=object)}

In [266]:
allergy_csv = pd.DataFrame([{"allergy": allergy, "allergy_id": hash(allergy) % 10000} for allergy in allergies_to_consider])

allergy_csv

,allergy,allergy_id
0,nut,7881
1,lactose,2286


In [267]:
allergens_list = []
for allergy in allergies_to_consider:
  keyword_list = np.append(allergen_keywords[allergy], allergy)
  allergens_list.extend([{"allergy_id": hash(allergy) % 10000, "ingredient_id": ingred_id} for ingred_id in ingredients_csv[ingredients_csv["Cleaned_Desc"].str.contains('|'.join(keyword_list), regex=True)]["NDB_No"]])

allergens_csv = pd.DataFrame(allergens_list)

allergens_csv

,allergy_id,ingredient_id
0,7881,2025
1,7881,4042
2,7881,4528
3,7881,4532
4,7881,11485
...,...,...
78,2286,18373
79,2286,19095
80,2286,22899
81,2286,42291


## Output

In [268]:
recipes_csv.to_csv('./cleaned_data/recipes.csv', index=False)
ingredients_csv.to_csv('./cleaned_data/ingredients.csv', index=False)
steps_csv.to_csv('./cleaned_data/steps.csv', index=False)
ingredient_in_csv.to_csv('./cleaned_data/ingredient_in.csv', index=False)
conversions_csv.to_csv('./cleaned_data/conversions.csv', index=False)
interactions_csv.to_csv('./cleaned_data/interactions.csv', index=False)
users_csv.to_csv('./cleaned_data/users.csv', index=False)
allergy_csv.to_csv('./cleaned_data/allergies.csv', index=False)
allergens_csv.to_csv('./cleaned_data/allergens.csv', index=False)